## Species Index for the SECAS Blueprint
The source data is provided by SARP. The following cells provide the final processing steps needed to be fed into SECAS BP (SA and MiddleSE):

- New attribute field added to the SECAS SGCN layer: Reclassed into bins based on count
- Reclassed SGCN layer converted to a raster based on the new field created

In [ ]:
#import libraries
import arcpy
import numpy as np
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [ ]:
#create variables from needed layers: SECAS States SGCN and EPA Floodplain
#EPA floodplain layer preprocessed from BRCI Model, consists of a polygon version for the SECAS Geography
epa_fp = r"D:\PRAAT\Aquatics\Species\Species_Indices\SpeciesIndex6421.gdb\floodplain_only"
sgcn = r"D:\PRAAT\Aquatics\Species\Species_Indices\SpeciesIndex6421.gdb\SECAS_SGCN"

In [ ]:
#convert featureclass to a dataframe
sgcn_df = pd.DataFrame.spatial.from_featureclass(location = sgcn)

In [ ]:
#create a variable for 
cols = sgcn_df.columns.tolist()

In [ ]:
#create a binned column based on species count established by the SA Team, this may be changed to 0-3 to match MiddleSE 
vals = [0,1,2,3,4]
reclass = [
    (sgcn_df[cols[24]] == 0),
    (sgcn_df[cols[24]] == 1),
    (sgcn_df[cols[24]] == 2),
    (sgcn_df[cols[24]] == 3),
    (sgcn_df[cols[24]] >= 4)
]

sgcn_df['reclassed'] = np.select(reclass,vals)

In [ ]:
#these are the fields want to keep and export back to create the feature class
cols_export = ['HUC12_1','reclassed','SHAPE']

In [ ]:
#set output path and name
out1 = r"D:\PRAAT\Aquatics\Species\Species_Indices\SpeciesIndex6421.gdb\reclassed_sgcn"

In [ ]:
#convert back to featureclass
out1 = r"D:\PRAAT\Aquatics\Species\Species_Indices\SpeciesIndex6421.gdb\reclassed_sgcn"
sgcn_df[cols_export].spatial.to_featureclass(out1)

In [ ]:
#convert new feature class to raster, mask extent to the EPA Floodplains
inFeatures = out1
valField = cols_export[1]
outRaster = r"D:\PRAAT\Aquatics\Species\Species_Indices\SpeciesIndex6421.gdb\sgcn_reclass_rast"
assignmentType = "MAXIMUM_AREA"
cellSize = 0.005
priorityField = ""

arcpy.PolygonToRaster_conversion(inFeatures, valField, outRaster, 
                                 assignmentType, priorityField, cellSize)
outR = r"D:\PRAAT\Aquatics\Species\Species_Indices\SpeciesIndex6421.gdb\sgcn_reclass_rast_fp"
outExtractByMask = arcpy.sa.ExtractByMask(outRaster, epa_fp); outExtractByMask.save(outR)